In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/UT_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4440597205,UT,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,4.903511e+14
1,5333267308,UT,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,4.901113e+14
2,7850817707,UT,"1055 North 500 West, Provo, UT",40.248562,-111.668402,4.904900e+14
3,8803828610,UT,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,4.903511e+14
4,9607187010,UT,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,4.903511e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4440597205,UT,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,4.903511e+14
1,5333267308,UT,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,4.901113e+14
2,7850817707,UT,"1055 North 500 West, Provo, UT",40.248562,-111.668402,4.904900e+14
3,8803828610,UT,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,4.903511e+14
4,9607187010,UT,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,4.903511e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
594,3842137205,UT,"1337 N 750 E, UT, UT",30.069203,-99.118273,NaN
8607,2720337203,UT,"136 E South Temple Suite 1900, SALT LAKE CITY, UT",40.768942,-111.887095,NaN
9202,5601547110,UT,"1750 E. Hwy 40, VERNAL, UT",39.001899,-92.733623,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('49')] #UT
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/3 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

3


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15426/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
594,3842137205,UT,"1337 N 750 E, UT, UT",30.069203,-99.118273,[]
8607,2720337203,UT,"136 E South Temple Suite 1900, SALT LAKE CITY, UT",40.768942,-111.887095,[490351140001000]
9202,5601547110,UT,"1750 E. Hwy 40, VERNAL, UT",39.001899,-92.733623,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_15426/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
594,3842137205,UT,"1337 N 750 E, UT, UT",30.069203,-99.118273,NaN
8607,2720337203,UT,"136 E South Temple Suite 1900, SALT LAKE CITY, UT",40.768942,-111.887095,490351140001000
9202,5601547110,UT,"1750 E. Hwy 40, VERNAL, UT",39.001899,-92.733623,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
594,3842137205,UT,"1337 N 750 E, UT, UT",30.069203,-99.118273,NaN
9202,5601547110,UT,"1750 E. Hwy 40, VERNAL, UT",39.001899,-92.733623,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
594,3842137205,NaN
8607,2720337203,490351140001000
9202,5601547110,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/UT_fips_scraped.csv") 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,884039,4440597205,UT,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,4.903511e+14
1,884040,5333267308,UT,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,4.901113e+14
2,884041,7850817707,UT,"1055 North 500 West, Provo, UT",40.248562,-111.668402,4.904900e+14
3,884042,8803828610,UT,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,4.903511e+14
4,884043,9607187010,UT,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,4.903511e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,884039,4440597205,UT,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,4.903511e+14,NaN
1,884040,5333267308,UT,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,4.901113e+14,NaN
2,884041,7850817707,UT,"1055 North 500 West, Provo, UT",40.248562,-111.668402,4.904900e+14,NaN
3,884042,8803828610,UT,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,4.903511e+14,NaN
4,884043,9607187010,UT,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,4.903511e+14,NaN
...,...,...,...,...,...,...,...,...
9253,893292,9048067208,UT,"676 LAVA POINTE DR #50, SAINT GEORGE, UT",37.076327,-113.599085,4.905327e+14,NaN
9254,893293,9342157005,UT,"61 E GORDON AVE, SALT LAKE CITY, UT",40.683543,-111.889893,4.903511e+14,NaN
9255,893294,9527478402,UT,"756 S 300 W, Salt Lake City, UT",40.752633,-111.900108,4.903511e+14,NaN
9256,893295,9736917000,UT,"412 E. 3200 N., LEHI, UT",40.427645,-111.842536,4.904901e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,884039,4440597205,UT,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,4.903511e+14,NaN,490351140002011.0
1,884040,5333267308,UT,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,4.901113e+14,NaN,490111270024066.0
2,884041,7850817707,UT,"1055 North 500 West, Provo, UT",40.248562,-111.668402,4.904900e+14,NaN,490490020013000.0
3,884042,8803828610,UT,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,4.903511e+14,NaN,490351126213007.0
4,884043,9607187010,UT,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,4.903511e+14,NaN,490351123023021.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4440597205,UT,"6219 S. Saunter Lane, SALT LAKE CITY, UT",40.760779,-111.891047,490351140002011.0
1,5333267308,UT,"450 East 1000 North, NORTH SALT LAKE, UT",40.859421,-111.900968,490111270024066.0
2,7850817707,UT,"1055 North 500 West, Provo, UT",40.248562,-111.668402,490490020013000.0
3,8803828610,UT,"9815 S Monroe St Ste 203, Sandy, UT",40.573544,-111.897991,490351126213007.0
4,9607187010,UT,"348 E WINCHESTER ST, SALT LAKE CITY, UT",40.632246,-111.880457,490351123023021.0
...,...,...,...,...,...,...
9253,9048067208,UT,"676 LAVA POINTE DR #50, SAINT GEORGE, UT",37.076327,-113.599085,490532715021002.0
9254,9342157005,UT,"61 E GORDON AVE, SALT LAKE CITY, UT",40.683543,-111.889893,490351116012016.0
9255,9527478402,UT,"756 S 300 W, Salt Lake City, UT",40.752633,-111.900108,490351140003025.0
9256,9736917000,UT,"412 E. 3200 N., LEHI, UT",40.427645,-111.842536,490490102163000.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
594,3842137205,UT,"1337 N 750 E, UT, UT",30.069203,-99.118273,NaN
9202,5601547110,UT,"1750 E. Hwy 40, VERNAL, UT",39.001899,-92.733623,NaN


Convert to File

In [19]:
fips_UT = fips_merge_drop

In [20]:
fips_UT.to_csv("../../../data/state_data/geo/geo_fips/23/UT_fips_scraped.csv") 